This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [18]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [19]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("data/mra.mha",itk.F)

In [20]:
res = ttk.ResampleImage[ImageType].New()
res.SetInput(im1)
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

In [27]:
xp1=[-3.9,-54.8,12.3]
xp2=[18.4,-16,-21]
xp3=[-46,-15,-13]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(False)
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetVerbose(True)
tube = vSeg.ExtractTubeInObjectSpace( xp1, 1 )
tube = vSeg.ExtractTubeInObjectSpace( xp2, 2 )
tube = vSeg.ExtractTubeInObjectSpace( xp3, 3 )
tubeG = vSeg.GetTubeGroup()

In [28]:
# Example of accessing data in a tube
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
r = list(map(lambda x: tube.GetPoint(x).GetRadiusInObjectSpace(), range(tube.GetNumberOfPoints())))

In [29]:
TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(im1)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()
view(outputImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [30]:
TTKImageMathType = ttk.ImageMath[ImageType,ImageType]

imMath = TTKImageMathType.New(Input = outputImage)
imMath.AddImages(im1, 2048, 1)
combinedImage = imMath.GetOutput()
view(combinedImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…